<a href="https://colab.research.google.com/github/priyal6/Rag-with-ollama/blob/main/qdrant_essentials.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qdrant_client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.2/377.2 kB 12.0 MB/s eta 0:00:00


In [ ]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.1 MB/s eta 0:00:00


In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
import numpy as np
from faker import Faker

In [ ]:
my_collection = "first_collection"

# Check if the collection exists and delete it if it does, to ensure a clean state for creation.
# This makes the cell re-runnable.
if my_collection in [collection.name for collection in qdrant_client.get_collections().collections]:
    print(f"Collection '{my_collection}' already exists. Deleting it first...")
    qdrant_client.delete_collection(collection_name=my_collection)
    print(f"Collection '{my_collection}' deleted.")

qdrant_client.create_collection(
    collection_name=my_collection,
    vectors_config = models.VectorParams(size=100, distance=models.Distance.COSINE)
)
print(f"Collection '{my_collection}' created successfully.")

Collection 'first_collection' already exists. Deleting it first...
Collection 'first_collection' deleted.
Collection 'first_collection' created successfully.


In [ ]:
data  = np.random.uniform(low=-1.0, high = 1.0, size=(1_000, 100))
index = list(range(1_000))

In [ ]:
qdrant_client.upsert(
    collection_name = my_collection,
    points = models.Batch(
        ids = index,
        vectors = data.tolist()
    )
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
qdrant_client.retrieve(
    collection_name = my_collection,
    ids=[10, 14, 500],
   # with_vectors=True
)

[Record(id=10, payload={}, vector=None, shard_key=None, order_value=None),
 Record(id=14, payload={}, vector=None, shard_key=None, order_value=None),
 Record(id=500, payload={}, vector=None, shard_key=None, order_value=None)]

In [ ]:
fake_something = Faker()
fake_something.name(), fake_something.address()

('Margaret Goodman', '4539 Morris Cliffs Suite 088\nMelanieport, VI 67046')

In [ ]:
payload = []
for i in range(1_000):
  payload.append(
      {
          "artist": fake_something.name(),
          "song": " ".join(fake_something.words()),
          "url_song": fake_something.url(),
          "year": fake_something.year(),
          "country": fake_something.country(),
      }
  )

payload[:3]

[{'artist': 'Catherine Coleman',
  'song': 'central make computer',
  'url_song': 'https://cooper-lane.net/',
  'year': '2022',
  'country': 'Nigeria'},
 {'artist': 'Shelby Robinson',
  'song': 'five Congress lose',
  'url_song': 'https://pope-powell.com/',
  'year': '2011',
  'country': 'Iraq'},
 {'artist': 'Desiree Caldwell',
  'song': 'way consumer every',
  'url_song': 'http://www.rogers-mccann.com/',
  'year': '1980',
  'country': 'French Guiana'}]

In [ ]:
qdrant_client.upsert(
    collection_name = my_collection,
    points = models.Batch(
        ids = index,
        vectors = data.tolist(),
        payloads = payload
    )
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
##semantic search
living_la_vida_loca = np.random.uniform(low=-1.0, high=1.0, size=(100)).tolist()
living_la_vida_loca[:5]

[-0.7938312623057258,
 -0.4375605282403727,
 0.0973573861734569,
 0.9918680147220973,
 -0.469288659062681]

In [ ]:
qdrant_client.query_points(
    collection_name=my_collection,
    query=living_la_vida_loca,
    limit=2
)

QueryResponse(points=[ScoredPoint(id=986, version=2, score=0.27957717, payload={'artist': 'Dominic Mcintyre', 'song': 'price method rather', 'url_song': 'http://www.smith.biz/', 'year': '1988', 'country': 'Aruba'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=928, version=2, score=0.2776755, payload={'artist': 'Karen Jones', 'song': 'save state foot', 'url_song': 'http://buck.com/', 'year': '1994', 'country': 'Luxembourg'}, vector=None, shard_key=None, order_value=None)])

In [ ]:
from qdrant_client.models import Distance, VectorParams

qdrant_client.create_collection(
    collection_name="test_collection",
    vectors_config=VectorParams(size=4, distance=Distance.DOT)
)

True

In [ ]:
from qdrant_client.models import PointStruct


operation_info = qdrant_client.upsert(
    collection_name="test_collection",
    wait=True,
    points= [
        PointStruct(id=1, vector=[0.05, 0.61, 0.76, 0.74], payload={"city": "Berlin"}),
        PointStruct(id=2, vector=[0.19, 0.81, 0.75, 0.11], payload={"city": "London"}),
        PointStruct(id=3, vector=[0.36, 0.55, 0.47, 0.94], payload={"city": "Moscow"}),
        PointStruct(id=4, vector=[0.18, 0.01, 0.85, 0.80], payload={"city": "New York"}),
        PointStruct(id=5, vector=[0.24, 0.18, 0.22, 0.44], payload={"city": "Beijing"}),
        PointStruct(id=6, vector=[0.35, 0.08, 0.11, 0.44], payload={"city": "Mumbai"}),

    ],
)

print(operation_info)

operation_id=1 status=<UpdateStatus.COMPLETED: 'completed'>


In [ ]:
search_result = qdrant_client.query_points(
    collection_name="test_collection",
    query=[0.2, 0.1, 0.9, 0.7],
    with_payload=True,
    limit=3
).points

print(search_result)

[ScoredPoint(id=4, version=1, score=1.362, payload={'city': 'New York'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=1, version=1, score=1.273, payload={'city': 'Berlin'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=3, version=1, score=1.208, payload={'city': 'Moscow'}, vector=None, shard_key=None, order_value=None)]
